# Analyze consensus from deep sequencing pileup versus Genbank accessions

Import Python modules:

In [ ]:
import altair as alt

import Bio.AlignIO

import pandas as pd

Get the pileup-consensus / Genbank alignments along with the descriptions from `snakemake`:

In [ ]:
alignments = snakemake.input.alignments
descriptors = snakemake.params.descriptors
output_csv = snakemake.output.csv
output_chart = snakemake.output.chart
output_mismatches = snakemake.output.mismatches

Create a data frame with the alignments, and then process it to get:
 - total sites in alignment
 - sites that are identical
 - sites that are ambiguous (a `N` in one sequence)
 - sites that are gapped in either sequence but have a called identity in other
 - sites that are mismatched called nucleotides
 - mismatch mutations in sequential 1, 2, ... numbering of reference

In [ ]:
assert len(alignments) == len(descriptors)

alignments_df = (
    pd.DataFrame({'sample': [d['sample'] for d in descriptors],
                  'genome': [d['genome'] for d in descriptors],
                  'aligner': [d['aligner'] for d in descriptors],
                  'alignment_file': alignments,
                  })
    )

def alignment_stats(aln_file):
    aln = Bio.AlignIO.read(aln_file, 'fasta')
    assert len(aln) == 2
    s1 = str(aln[0].seq).upper()
    s2 = str(aln[1].seq).upper()
    assert len(s1) == len(s2)
    total_sites = len(s1)
    identical_sites = ambiguous_sites = mismatched_sites = gapped_sites = r = 0
    muts = []
    for nt1, nt2 in zip(s1, s2):
        if nt1 != '-':
            r += 1
        if nt1 == nt2:
            identical_sites += 1
        elif nt1 == 'N' or nt2 == 'N':
            ambiguous_sites += 1
        elif nt1 == '-' or nt2 == '-':
            gapped_sites += 1
        else:
            mismatched_sites += 1
            muts.append(f"{nt1}{r}{nt2}")
    assert total_sites == identical_sites + ambiguous_sites + mismatched_sites + gapped_sites
    return total_sites, identical_sites, ambiguous_sites, mismatched_sites, gapped_sites, ', '.join(muts)

for col, vals in zip(['total', 'identical', 'ambiguous', 'mismatched',
                      'gapped', 'mismatches (consensus -> Genbank)'],
                     zip(*alignments_df['alignment_file'].map(alignment_stats))
                     ):
    alignments_df[col] = vals
    
alignments_df = (
    alignments_df
    .drop(columns='alignment_file')
    .assign(sample=lambda x: pd.Categorical(x['sample'], x['sample'].unique(), ordered=True),
            genome=lambda x: pd.Categorical(x['genome'], x['genome'].unique(), ordered=True),
            aligner=lambda x: pd.Categorical(x['aligner'], x['aligner'].unique(), ordered=True),
            )
    .sort_values(['sample', 'genome', 'aligner'])
    .reset_index(drop=True)
    )

print(f"Saving statistics to {output_csv}")
alignments_df.to_csv(output_csv, index=False)

alignments_df

Get intersection of mismatches among aligners:

In [ ]:
assert 'intersection all aligners' not in alignments_df['aligner'].unique()

mismatches = (
    alignments_df
    .groupby(['sample', 'genome'], as_index=False)
    .aggregate({'mismatches (consensus -> Genbank)':
                lambda g: ', '.join(
                        tup[1] for tup in
                            sorted([(int(mut[1: -1]), mut) for mut in
                                    set.intersection(*[set(mismatches) for mismatches
                                                       in g.str.split(', ')])
                                    if mut
                                    ])
                        )
                })
    .assign(aligner='intersection all aligners')
    .append(alignments_df)
    [['sample', 'genome', 'aligner', 'mismatches (consensus -> Genbank)']]
    .assign(aligner=lambda x: pd.Categorical(x['aligner'], x['aligner'].unique(), ordered=True))
    .sort_values(['sample', 'genome', 'aligner'])
    .reset_index(drop=True)
    )

print(f"Writing to {output_mismatches}")
mismatches.to_csv(output_mismatches, index=False)

mismatches

Build an Altair interactive chart plotting data frame above:

In [ ]:
selection_genome = alt.selection_single(
            name='reference',
            fields=['genome'],
            bind=alt.binding_select(options=alignments_df['genome'].unique().tolist()),
            )

selection_aligner = alt.selection_single(
            name='read',
            fields=['aligner'],
            bind=alt.binding_select(options=[None] + list(alignments_df['aligner'].unique()),
                                    labels=['all'] + list(alignments_df['aligner'].unique()))
            )

selection_sample = alt.selection_single(
            name='virus',
            fields=['sample'],
            bind=alt.binding_select(options=[None] + list(alignments_df['sample'].unique()),
                                    labels=['all'] + list(alignments_df['sample'].unique()))
            )

xmax = alignments_df['identical'].max()
ymax = alignments_df['mismatched'].max()

chart = (
    alt.Chart(alignments_df)
    .encode(
        x=alt.X('identical:Q',
                axis=alt.Axis(tickMinStep=1),
                scale=alt.Scale(domain=(-0.02 * xmax, 1.02 * xmax),
                                nice=False)
                ),
        y=alt.Y('mismatched:Q',
                axis=alt.Axis(tickMinStep=1),
                scale=alt.Scale(domain=(-0.02 * ymax, 1.02 * ymax),
                                nice=False)
                ),
        color='aligner:N',
        fill='aligner:N',
        shape='aligner:N',
        tooltip=['sample',
                 'aligner',
                 'identical',
                 'ambiguous',
                 'mismatched',
                 'gapped',
                 'mismatches (consensus -> Genbank)']
        )
    .mark_point(opacity=0.5,
                size=40,
                )
    .add_selection(selection_genome,
                   selection_aligner,
                   selection_sample,
                   )
    .transform_filter(selection_genome)
    .transform_filter(selection_aligner)
    .transform_filter(selection_sample)
    .interactive()
    .configure_axis(grid=False)
    .properties(width=300,
                height=300,
                title='Deep sequencing consensus compared to Genbank',
                )
    )

print(f"Saving chart to {output_chart}")
chart.save(output_chart)

chart